In [58]:
import os
import pandas as pd
from datetime import timedelta
from breeze_connect import BreezeConnect

In [74]:
breeze = BreezeConnect(api_key=")u0t936807l9Y56X01z5986K20e@2R83")

breeze.generate_session(
    api_secret="n31904415e6b623f3353*(H584w1pV45",
    session_token="54483088"
)

In [76]:

def fetch_breeze_daily(symbol, start_date):
    all_dfs = []

    start_date = pd.to_datetime(start_date)
    today = pd.Timestamp.today()

    # Iterate year by year
    for year in range(start_date.year, today.year + 1):
        year_start = max(start_date, pd.Timestamp(year=year, month=1, day=1))
        year_end = min(today, pd.Timestamp(year=year, month=12, day=31))

        from_date = year_start.strftime("%Y-%m-%dT09:15:00")
        to_date = (year_end + timedelta(days=1)).strftime("%Y-%m-%dT09:15:00")

        resp = breeze.get_historical_data_v2(
            interval=INTERVAL,
            from_date=from_date,
            to_date=to_date,
            stock_code=symbol,
            exchange_code="NSE",
            product_type="cash"
        )

        if resp.get("Status") == 200 and resp.get("Success"):
            df = pd.DataFrame(resp["Success"])
            df["datetime"] = pd.to_datetime(df["datetime"])
            all_dfs.append(df)

    if not all_dfs:
        return pd.DataFrame()

    # Combine all years
    final_df = (
        pd.concat(all_dfs, ignore_index=True)
        .drop_duplicates(subset=["datetime"])
        .sort_values("datetime")
        .reset_index(drop=True)
    )

    return final_df


In [78]:
symbols = [ "INFTEC"]
all_data = []

for symbol in symbols:
    
    from_date="2015-01-01T09:15:00"
    to_date="2026-01-31T15:30:00"

    df = fetch_breeze_daily(symbol, from_date)
    df["symbol"] = symbol
    all_data.append(df)

final_df = pd.concat(all_data)
print(final_df.head())
final_df.to_excel("../out/equity/test.xlsx", index=False)

    close   datetime exchange_code    high     low    open stock_code  \
0  504.49 2015-01-02           NSE  504.76  493.00  493.00     INFTEC   
1  499.00 2015-01-05           NSE  507.50  494.38  502.48     INFTEC   
2  491.25 2015-01-06           NSE  496.25  483.53  495.00     INFTEC   
3  491.51 2015-01-07           NSE  493.69  487.50  491.25     INFTEC   
4  494.81 2015-01-08           NSE  499.25  487.50  496.40     INFTEC   

     volume  symbol  
0   6716224  INFTEC  
1   9919872  INFTEC  
2   9646456  INFTEC  
3   7183004  INFTEC  
4  13539880  INFTEC  


In [15]:
# --------------------------------------------------
# CONFIG
# --------------------------------------------------
MDM_FILE = "../data/normalized/mdm.xlsx"
OUT_DIR = "../out/equityhistory"
INTERVAL = "1day"

ISIN_COL = "isin"
CODE_COL = "icici breeze code"
COMPANY_COL = "company"
DATE_COL = "trade date"

In [17]:
# --------------------------------------------------
# LOAD EXCEL
# --------------------------------------------------
equity_df = pd.read_excel(MDM_FILE, sheet_name="Equity")
txn_df = pd.read_excel(MDM_FILE, sheet_name="Equity Transaction")

In [19]:
equity_df.head()

,ISIN,Company,NSE Symbol,BSE Code,Incorporation Date,ISIN Creation Date,ICICI Breeze Code
0,INE238A01034,Axis Bank Ltd,AXISBANK,532215,03-Dec-1993,NaT,AXIBAN
1,INE457A01014,Bank of Maharashtra,MAHABANK,532525,16-Sep-1935,NaT,BANMAH
2,INE323A01026,Bosch Ltd,BOSCHLTD,500530,12-Nov-1951,NaT,BOSLIM
3,INE925R01014,CMS Info Systems Ltd,CMSINFO,543441,26-Mar-2008,NaT,CMSIN
4,INE335K01011,Coffee Day Enterprises Ltd,COFFEEDAY,539436,20-Jun-2008,NaT,COFDAY


In [21]:
txn_df.head()

,Person,Account,Trade Date,ISIN,Security,Segment,Action,Quantity,Price/Share,Net Amount,BrokeragePerUnit,NetBrokerage,STT,Stamp Duty,GST,Total Amount
0,Arun Venkatesan,ICICI,2020-04-28,INE090A01021,ICICI BANK LTD.,CASH,Buy,20,353.75,7075.0,1.25,25.0,7.00,0.45,4.52,7111.97
1,Arun Venkatesan,ICICI,2020-07-06,INE040A01034,HDFC BANK LTD,CASH,Buy,10,1100.00,11000.0,6.05,60.5,11.00,2.00,10.95,11084.45
2,Arun Venkatesan,ICICI,2021-12-20,INE562A01011,INDIAN BANK,CASH,Buy,62,136.45,8459.9,0.75,46.5,0.00,0.00,0.00,8506.40
3,Arun Venkatesan,ICICI,2021-12-20,INE562A01011,INDIAN BANK,CASH,Buy,38,136.45,5185.1,0.75,28.5,13.99,2.00,13.55,5243.14
4,Arun Venkatesan,ICICI,2021-12-23,INE040A01034,HDFC BANK LTD,CASH,Buy,10,1450.00,14500.0,7.97,79.7,14.40,2.18,14.40,14610.68


In [23]:
# Normalize column names
equity_df.columns = equity_df.columns.str.strip().str.lower()
txn_df.columns = txn_df.columns.str.strip().str.lower()

# Normalize dates
txn_df[DATE_COL] = pd.to_datetime(txn_df[DATE_COL])

In [41]:
# --------------------------------------------------
# JOIN USING ISIN
# --------------------------------------------------
min_trade_dates = (
    txn_df.groupby(ISIN_COL)[DATE_COL]
    .min()
    .reset_index()
    .rename(columns={DATE_COL: "start_date"})
)


In [43]:
min_trade_dates.head()

,isin,start_date
0,INE003A01024,2020-03-26
1,INE018A01030,2020-03-27
2,INE155A01022,2021-06-08
3,INE335K01011,2021-06-09
4,INE457A01014,2021-06-09


In [45]:
symbols_df = (
    equity_df
    .merge(min_trade_dates, on=ISIN_COL, how="inner")
    [[ISIN_COL, COMPANY_COL, CODE_COL, "start_date"]]
)
symbols_df.head()

,isin,company,icici breeze code,start_date
0,INE238A01034,Axis Bank Ltd,AXIBAN,2023-02-27
1,INE457A01014,Bank of Maharashtra,BANMAH,2024-12-18
2,INE323A01026,Bosch Ltd,BOSLIM,2020-03-27
3,INE335K01011,Coffee Day Enterprises Ltd,COFDAY,2024-12-16
4,INE0KL801015,DCX System Ltd,DCXSYS,2024-08-21


In [47]:
# --------------------------------------------------
# HELPERS
# --------------------------------------------------
def existing_trade_dates(file_path):
    if not os.path.exists(file_path):
        return set()
    df = pd.read_excel(file_path)
    if "datetime" not in df.columns:
        return set()
    return set(pd.to_datetime(df["datetime"]).dt.date)


def fetch_breeze_daily(symbol, start_date):
    from_date = start_date.strftime("%Y-%m-%dT09:15:00")
    to_date = (pd.Timestamp.today() + timedelta(days=1)).strftime(
        "%Y-%m-%dT09:15:00"
    )

    resp = breeze.get_historical_data_v2(
        interval=INTERVAL,
        from_date=from_date,
        to_date=to_date,
        stock_code=symbol,
        exchange_code="NSE",
        product_type="cash"
    )

    if resp.get("Status") == 200 and resp.get("Success"):
        df = pd.DataFrame(resp["Success"])
        df["datetime"] = pd.to_datetime(df["datetime"])
        return df

    return pd.DataFrame()

In [49]:
os.makedirs(OUT_DIR, exist_ok=True)

In [55]:
# --------------------------------------------------
# MAIN LOOP
# --------------------------------------------------
for _, row in symbols_df.iterrows():
    isin = row[ISIN_COL]
    company = row[COMPANY_COL].replace("/", "-")
    symbol = row[CODE_COL]
    start_date = row["start_date"].date()

    out_file = os.path.join(
        OUT_DIR, f"{isin}-{company}.xlsx"
    )

    pulled_dates = existing_trade_dates(out_file)

    print(f"Processing {company} ({isin}) from {start_date}")

    df = fetch_breeze_daily(symbol, start_date)

    if df.empty:
        print("  No data returned")
        continue

    df["trade_date"] = df["datetime"].dt.date
    df["isin"] = isin
    df["company"] = company
    df["symbol"] = symbol
    df["source"] = "BREEZE"

    # Exclude already pulled dates
    df_new = df[~df["trade_date"].isin(pulled_dates)]

    if df_new.empty:
        print("  No new rows to append")
        continue

    if os.path.exists(out_file):
        old_df = pd.read_excel(out_file)
        final_df = pd.concat([old_df, df_new], ignore_index=True)
    else:
        final_df = df_new

    final_df.sort_values("datetime", inplace=True)
    final_df.to_excel(out_file, index=False)

    print(f"  Saved {len(df_new)} rows")

print("✅ Equity historical extraction completed.")

Processing Axis Bank Ltd (INE238A01034) from 2023-02-27
  No new rows to append
Processing Bank of Maharashtra (INE457A01014) from 2024-12-18
  No new rows to append
Processing Bosch Ltd (INE323A01026) from 2020-03-27
  No new rows to append
Processing Coffee Day Enterprises Ltd (INE335K01011) from 2024-12-16
  No new rows to append
Processing DCX System Ltd (INE0KL801015) from 2024-08-21
  Saved 3 rows
Processing GAIL (INDIA) Ltd (INE129A01019) from 2024-08-20
  No new rows to append
Processing HDFC Bank Ltd (INE040A01034) from 2020-06-10
  No new rows to append
Processing ICICI Bank Ltd (INE090A01021) from 2020-04-28
  No new rows to append
Processing IDFC First Bank Ltd (INE092T01019) from 2022-09-19
  No new rows to append
Processing Indian Bank (INE562A01011) from 2021-12-20
  No new rows to append
Processing Indian Hotels Company Ltd (INE053A01029) from 2021-06-09
  No new rows to append
Processing Infosys Ltd (INE009A01021) from 2015-06-15
  No new rows to append
Processing IRCT